In [1]:
import numpy as np
import pandas as pd
import os
import pyrsgis as rg
import matplotlib.pyplot as plt
import rasterio
from osgeo import gdal
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
#%matplotlib qt

In [2]:
#band_path = 'F:\\Datasets\\satellite\\S2A_MSIL2A_20190104T051211_N0211_R019_T43PGQ_20190104T094623.SAFE\\GRANULE\\L2A_T43PGQ_A018463_20190104T052230\\IMG_DATA\\R10m\\'
#B2_jp2 = 'T43PGQ_20190104T051211_B0{}_10m.jp2'.format(2)
image_path = 'F:\\Datasets\\satellite\\S2A_MSIL2A_20190104T051211_N0211_R019_T43PGQ_20190104T094623.SAFE\\GRANULE\\L2A_T43PGQ_A018463_20190104T052230\\IMG_DATA\\R10m\\'
image_file = r'T43PGQ_20190104T051211_B02_10m.jp2'
#band2 = rasterio.open(image_path+image_file, driver='JP2OpenJPEG') #blue

In [3]:
src_ds = gdal.Open(image_path+image_file)


In [4]:
#myarray = np.array(ds.GetRasterBand(1).ReadAsArray())
print("[ RASTER BAND COUNT ]: ", src_ds.RasterCount)
for band in range( src_ds.RasterCount ):
    band += 1
    print("[ GETTING BAND ]: ", band)
    srcband = src_ds.GetRasterBand(band)
    if srcband is None:
        continue

    stats = srcband.GetStatistics( True, True )
    if stats is None:
        continue

    print("[ STATS ] =  Minimum=%.3f, Maximum=%.3f, Mean=%.3f, StdDev=%.3f" % ( \
                stats[0], stats[1], stats[2], stats[3] ))

[ RASTER BAND COUNT ]:  1
[ GETTING BAND ]:  1
[ STATS ] =  Minimum=0.000, Maximum=18128.000, Mean=547.464, StdDev=280.519


In [5]:
myarray = np.array(src_ds.GetRasterBand(1).ReadAsArray())


In [6]:
print(type(myarray))
print(myarray.shape)

<class 'numpy.ndarray'>
(10980, 10980)


In [7]:
np.isnan(myarray).any()

False

In [8]:
if False:
    std_scaler = StandardScaler()
    standardized_data = std_scaler.fit_transform(myarray)

    minmax_scaler = MinMaxScaler()
    minmax_data = minmax_scaler.fit_transform(myarray)


    l1_norm_data = normalize(myarray, norm='l1')
    l2_norm_data = normalize(myarray, norm='l2')

In [9]:
gt_file = r'\data\water_mask.npy'
gt_path = r'F:\workspace\jupyterNotebooks\Remote Sensing\ISRO'

In [10]:
mask  = np.load(gt_path+gt_file)

In [11]:
print(mask.shape)
print(type(mask))

(10980, 10980)
<class 'numpy.ndarray'>


In [12]:
if False:
    plt.imshow(mask)
    plt.show()

In [13]:
myarray = myarray.flatten()
mask = mask.flatten()
#print(X.shape, Y.shape)
df = pd.DataFrame({'X':myarray, 
                    'Y':mask})
print(df.head())

   X  Y
0  0  0
1  0  0
2  0  0
3  0  0
4  0  0


In [14]:
df.drop(df[df.X == 0].index,inplace=True)


MemoryError: Unable to allocate 908. MiB for an array with shape (118954728,) and data type int64

In [ ]:
def split_stratified_into_train_val_test(df_input, stratify_colname='Y',
                                         frac_train=0.6, frac_val=0.15, frac_test=0.25,
                                         random_state=None):
    '''
    Splits a Pandas dataframe into three subsets (train, val, and test)
    following fractional ratios provided by the user, where each subset is
    stratified by the values in a specific column (that is, each subset has
    the same relative frequency of the values in the column). It performs this
    splitting by running train_test_split() twice.

    Parameters
    ----------
    df_input : Pandas dataframe
        Input dataframe to be split.
    stratify_colname : str
        The name of the column that will be used for stratification. Usually
        this column would be for the label.
    frac_train : float
    frac_val   : float
    frac_test  : float
        The ratios with which the dataframe will be split into train, val, and
        test data. The values should be expressed as float fractions and should
        sum to 1.0.
    random_state : int, None, or RandomStateInstance
        Value to be passed to train_test_split().

    Returns
    -------
    df_train, df_val, df_test :
        Dataframes containing the three splits.
    '''

    if frac_train + frac_val + frac_test != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' % \
                         (frac_train, frac_val, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X,
                                                          y,
                                                          stratify=y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    # Split the temp dataframe into val and test dataframes.
    relative_frac_test = frac_test / (frac_val + frac_test)
    df_val, df_test, y_val, y_test = train_test_split(df_temp,
                                                      y_temp,
                                                      stratify=y_temp,
                                                      test_size=relative_frac_test,
                                                      random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val) + len(df_test)

    return df_train, df_val, df_test

In [ ]:
df_train, df_val, df_test = split_stratified_into_train_val_test(df,stratify_colname='Y',
                                         frac_train=0.6, frac_val=0.15, frac_test=0.25,
                                         random_state=None)
X_train, y_train = df_train.X, df_train.Y
X_test, y_test = df_test.X, df_test.Y
X_val, y_val = df_val.X, df_val.Y